# imports

In [ ]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import os
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from geopy import distance
import math

from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint

# Data Load

In [ ]:
#base_path =  "./dataset/"
base_path = "../input/nyc-taxi-trip-duration/"

In [ ]:
os.listdir(base_path)

# Data Exploration

In [ ]:
data_df = pd.read_csv(base_path+"train.csv")

In [ ]:
data_df.head()

In [ ]:
data_df.describe()

### vendor_id

In [ ]:
sns.boxplot(x='vendor_id', y='trip_duration', data=data_df)

Data point bigger then 500000

In [ ]:
data_df[(data_df['trip_duration']>500000)].count()

**Therefore permanently droppping these rows**

In [ ]:
data_df.drop(data_df[data_df['trip_duration'] > 500000].index, inplace=True)

### Distribution of trip_duration

In [ ]:
sns.distplot(data_df['trip_duration'], bins=20000)

Since it is left skewed, so we will take log(with any base) of it.

In [ ]:
sns.distplot(np.log(data_df['trip_duration']), bins=20000)

In [ ]:
data_df['trip_duration'] = data_df['trip_duration'].apply(math.log)

### vendor_id

In [ ]:
data_df = data_df[(data_df['trip_duration']<100000)]

In [ ]:
data_df.reset_index(drop = True, inplace=True)

In [ ]:
sns.countplot(x = 'vendor_id', data=data_df)

In [ ]:
sns.boxplot(x='vendor_id', y='trip_duration', data=data_df)

From the above graph we can see **vendor_id** can play a key role

### passenger_count

In [ ]:
sns.countplot(x = 'passenger_count', data=data_df)

In [ ]:
sns.boxplot(x = 'passenger_count', y = 'trip_duration', data=data_df)

**passenger_count** is an important feature

### store_and_fwd_flag

In [ ]:
sns.countplot(x = 'store_and_fwd_flag', data=data_df)

In [ ]:
data_df[data_df['store_and_fwd_flag']=='Y']['trip_duration'].describe()

In [ ]:
sns.distplot(data_df[data_df['store_and_fwd_flag']=='Y']['trip_duration'], bins=1000)

In [ ]:
data_df[data_df['store_and_fwd_flag']=='N']['trip_duration'].describe()

In [ ]:
sns.distplot(data_df[data_df['store_and_fwd_flag']=='N']['trip_duration'], bins=1000)

In [ ]:
sns.boxplot(x = 'store_and_fwd_flag', y = 'trip_duration', data=data_df)

### Time

In [ ]:
def strtodatetime(x):
    return datetime.strptime(x, "%Y-%m-%d %H:%M:%S")

In [ ]:
data_df['pickup_datetime'] = data_df['pickup_datetime'].apply(strtodatetime)

Only considering pickup time foe day of week

In [ ]:
def dayofweek(x):
    return x.weekday()

In [ ]:
data_df['day_of_week'] = data_df['pickup_datetime'].apply(dayofweek)

In [ ]:
sns.boxenplot(x='day_of_week', y='trip_duration', data=data_df)

Will see with and without this feature.

### Distance

In [ ]:
distance.distance((data_df['pickup_latitude'].iloc[0],
                  data_df['pickup_longitude'].iloc[0]),
                 (data_df['dropoff_latitude'].iloc[0],
                 data_df['dropoff_longitude'].iloc[0])).m

In [ ]:
def dist(x):
   return distance.distance((x[0], x[1]),
                 (x[2], x[3])).m

In [ ]:
data_df['dist'] = data_df[['pickup_latitude',
         'pickup_longitude',
         'dropoff_latitude',
         'dropoff_longitude']].apply(lambda x:dist(x), axis=1)

In [ ]:
sns.distplot(data_df['dist'], bins=1000)

In [ ]:
data_df[(data_df['pickup_latitude']==data_df['dropoff_latitude']) &
        (data_df['pickup_longitude']==data_df['dropoff_longitude'])].count()

There are data with same pickup and drop location, so we will replace dist 0 with 1, and then take log of it.

In [ ]:
#data_df.drop(data_df[(data_df['pickup_latitude']==data_df['dropoff_latitude']) &
#        (data_df['pickup_longitude']==data_df['dropoff_longitude'])].index,
#            inplace=True)

In [ ]:
data_df['dist'].replace(to_replace=0, value=1, inplace=True)

In [ ]:
#log
data_df['dist'] = data_df['dist'].apply(math.log)

In [ ]:
sns.distplot(data_df['dist'], bins=1000)

### Time Conversion

In [ ]:
#currently handling data only day wise
MAX_SECONDS_IN_DAY = 24*60*60
def timetosectosincosday(x):
    #x = datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
    initial_date = datetime(x.year, x.month, x.day, 0 , 0, 0) # 1 is for day
    sec = (x-initial_date).total_seconds()
    sin = math.sin(2*math.pi*(sec/MAX_SECONDS_IN_DAY))
    cos = math.cos(2*math.pi*(sec/MAX_SECONDS_IN_DAY))
    return sin, cos

In [ ]:
data_df['pickup_sin_sec'] ,data_df['pickup_cos_sec'] = zip(*data_df['pickup_datetime'].map(timetosectosincosday))

In [ ]:
#data day of week
MAX_DAY_IN_WEEK = 6 # from 0-6
def dayofweektosincosday(day):
    #x = datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
    sin = math.sin(2*math.pi*(day/MAX_DAY_IN_WEEK))
    cos = math.cos(2*math.pi*(day/MAX_DAY_IN_WEEK))
    return sin, cos

In [ ]:
data_df['sin_day'] ,data_df['cos_day'] = zip(*data_df['day_of_week'].map(dayofweektosincosday))

# Normalize

In [ ]:
df_processed =  data_df[['vendor_id',
                        'passenger_count',
                        'pickup_sin_sec',
                        'pickup_cos_sec',
                        'pickup_longitude',
                        'pickup_latitude',
                        'dropoff_longitude',
                        'dropoff_latitude',
                        'sin_day',
                        'cos_day',
                        'dist',
                        'trip_duration']]

In [ ]:
X = data_df[['vendor_id',
            'passenger_count',
            'pickup_sin_sec',
            'pickup_cos_sec',
            'pickup_longitude',
            'pickup_latitude',
            'dropoff_longitude',
            'dropoff_latitude',
            'sin_day',
            'cos_day',
            'dist']]

In [ ]:
y = data_df[['trip_duration']]

In [ ]:
standardScalarX = StandardScaler().fit(X)
X  = standardScalarX.transform(X)

In [ ]:
standardScalarY = StandardScaler().fit(y)
y = standardScalarY.transform(y)

In [ ]:
X.shape, y.shape

# Model

In [ ]:
model = Sequential()
model.add(Dense(128, activation = 'relu',input_shape=(X.shape[1],)))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1))

In [ ]:
model.compile(optimizer=Adam(lr=0.00001),
              metrics=['mean_squared_error'], 
              loss='mean_squared_error')

In [ ]:
#filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='mean_squared_error', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
history = model.fit(X, y,
                   batch_size=32,
                   validation_split=0.2,
                   epochs=50, 
                   callbacks=callbacks_list)

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Model Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["train", "validation"])
plt.savefig('loss.png')

In [ ]:
model.save('model.hdf5')

# Test Set

In [ ]:
os.listdir(base_path)

In [ ]:
test_df = pd.read_csv(base_path+"test.csv")

In [ ]:
test_df.head()

Feature Used
**['vendor_id',
            'passenger_count',
            'pickup_sin_sec',
            'pickup_cos_sec',
            'pickup_longitude',
            'pickup_latitude',
            'dropoff_longitude',
            'dropoff_latitude',
            'sin_day',
            'cos_day',
            'dist']**

In [ ]:
test_df['pickup_datetime'] = test_df['pickup_datetime'].apply(strtodatetime)

In [ ]:
test_df['day_of_week'] = test_df['pickup_datetime'].apply(dayofweek)

In [ ]:
test_df['sin_day'] ,test_df['cos_day'] = zip(*test_df['day_of_week'].map(dayofweektosincosday))

In [ ]:
test_df['dist'] = test_df[['pickup_latitude',
         'pickup_longitude',
         'dropoff_latitude',
         'dropoff_longitude']].apply(lambda x:dist(x), axis=1)

In [ ]:
test_df['dist'].replace(to_replace=0, value=1, inplace=True)

In [ ]:
test_df['dist'] = test_df['dist'].apply(math.log)

In [ ]:
test_df['pickup_sin_sec'] ,test_df['pickup_cos_sec'] = zip(*test_df['pickup_datetime'].map(timetosectosincosday))

In [ ]:
test_df_processed =  test_df[['vendor_id',
                            'passenger_count',
                            'pickup_sin_sec',
                            'pickup_cos_sec',
                            'pickup_longitude',
                            'pickup_latitude',
                            'dropoff_longitude',
                            'dropoff_latitude',
                            'sin_day',
                            'cos_day',
                            'dist']]

In [ ]:
test_data =  standardScalarX.transform(test_df_processed)

In [ ]:
y_pred = model.predict(test_data)

In [ ]:
y_pred =standardScalarY.inverse_transform(y_pred)
y_pred = np.exp(y_pred)

In [ ]:
result = test_df[['id']]

In [ ]:
result['trip_duration'] = pd.DataFrame(data=y_pred, columns=['trip_duration'])

In [ ]:
result.head()

In [ ]:
result.to_csv("submission.csv", index = False)